In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "lib/python"))

import math
import numpy as np
import numpy.polynomial.polynomial as poly

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader
from picopic.plain_reader import PlainReader

In [15]:
##  configuration options
data_path = '../'

timestamp=4e-11
show_grid=False
use_cache=False
cmap='terrain'
image_interpolation = 'nearest'
autoselect = True
ylim=None
cmp = 'velocity' # component
specie = 'electrons'

# separate all velocity range to N equal segments
# to compare the number of particles in each of them
segments = 500

use_approx=True # use approximation
approx_degree=18 # degree of polynomial approximation
use_directed_filter=True

longitude=0.025

if os.path.isfile(os.path.join(data_path, "metadata.json")):
    reader = PlainReader(path = data_path, use_cache=use_cache, verbose=True)
elif os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, use_cache=use_cache, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
# size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]
size=[0, 0, 64, 64]
# size=[0,200,64,300]

r_scale = (size[2] - size[0]) / reader.meta.geometry_grid[0]
z_scale = (size[3] - size[1]) / reader.meta.geometry_grid[1]

# x_axis_label = r'$\mathit{R (m)}$'
# y_axis_label = r'$\mathit{\rho (m^{-3})}$'

x_axis_label = r'$\mathit{t (s)}$'
y_r_axis_label = r'$\mathit{E_r (\\frac{V}{m})}$'
y_z_axis_label = r'$\mathit{E_z (\\frac{V}{m})}$'

ylim_r=None
ylim_z=None

cbar_axis_label = r'$\mathit{\rho (m^{-3})}$'

plot_name = r'$\mathbf{%s\enspace Density\enspace (\rho)}$' % specie.capitalize()

In [16]:
# get data
data_r = data_z = data_phi = []
pos_r = pos_z = []

for probe in reader.meta.probes:
    if probe.specie == specie:
        frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
        timestamp = frame * probe.schedule * reader.meta.time[2]
        if (probe.shape == 'rec') and (probe.size[0] == size[0]) and (probe.size[1] == size[1]) and(probe.size[2] == size[2]) and(probe.size[3] == size[3]):
            if probe.component == 'velocity':
                data_r = reader.get_frame("{}/{}".format('velocity', specie), size, "{}_0".format(frame))[:]
                data_phi = reader.get_frame("{}/{}".format('velocity', specie), size, "{}_1".format(frame))[:]
                data_z = reader.get_frame("{}/{}".format('velocity', specie), size, "{}_2".format(frame))[:]
            if probe.component == 'position':
                pos_r = reader.get_frame("{}/{}".format('position', specie), size, "{}_0".format(frame))[:]
                pos_z = reader.get_frame("{}/{}".format('position', specie), size, "{}_2".format(frame))[:]
        
data = np.sqrt(np.power(data_r, 2) + np.power(data_phi, 2) + np.power(data_z, 2))

Loading data set velocity/electrons/rec:0-64_0-64/16_0...done
Loading data set velocity/electrons/rec:0-64_0-64/16_1...done
Loading data set velocity/electrons/rec:0-64_0-64/16_2...done
Loading data set position/electrons/rec:0-64_0-64/16_0...done
Loading data set position/electrons/rec:0-64_0-64/16_2...done


In [17]:
if use_directed_filter:
    # substract directed velocity
    r_grid_amount = int((size[2]-size[0]) / 2)
    z_grid_amount = int((size[3]-size[1]) / 2)

    subtrahend_r = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    subtrahend_phi = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    subtrahend_z = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    subtrahend = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    count = np.ones(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    
    dr = reader.meta.geometry_size[0] / reader.meta.geometry_grid[0]
    dz = reader.meta.geometry_size[1] / reader.meta.geometry_grid[1]
    r_shift = size[0] * dr
    z_shift = size[1] * dz
    
    for i in range(0, r_grid_amount):
        for j in range(0, z_grid_amount):
            for n in range(0, len(pos_r)):
                if pos_r[n] > r_shift + i * dr and pos_r[n] <= r_shift + (i+1) * dr and pos_z[n] > z_shift + j * dz and pos_z[n] <= z_shift + (j+1) * dz:
                    subtrahend_r[i][j] += data_r[n]
                    subtrahend_phi[i][j] += data_phi[n]
                    subtrahend_z[i][j] += data_z[n]
                    count[i][j] += 1
    
    subtrahend_r /= count
    subtrahend_phi /= count
    subtrahend_z /= count
    
    subtrahend = np.sqrt(np.power(subtrahend_r, 2) + np.power(subtrahend_z, 2) + np.power(subtrahend_phi, 2))
       
    # substract directed velocity
    for i in range(0, len(data_r)):
        p_r = pos_r[i]
        p_z = pos_z[i]
        r_cell_number = math.floor((p_r - r_shift) / dr - 1)
        z_cell_number = math.floor((p_z - z_shift) / dz - 1)
        if r_cell_number > r_grid_amount - 1: r_cell_number = r_grid_amount - 1
        if z_cell_number > z_grid_amount - 1: z_cell_number = z_grid_amount - 1
        data_r[i] = data_r[i] - subtrahend_r[r_cell_number][z_cell_number]
        data_phi[i] = data_phi[i] - subtrahend_phi[r_cell_number][z_cell_number]
        data_z[i] = data_z[i] - subtrahend_z[r_cell_number][z_cell_number]
        # data[i] = data[i] - subtrahend[r_cell_number][z_cell_number]
    data = np.sqrt(np.power(data_r, 2) + np.power(data_phi, 2) + np.power(data_z, 2))

In [18]:
velocity_max = np.max(data)
velocity_min = np.min(data)

stat = np.zeros(segments)

slices = np.linspace(velocity_min, velocity_max, segments)

for i in data:
    for j in range(len(slices)-1):
        if i > slices[j] and i <= slices[j+1]:
            stat[j] += 1
            
# stat = stat[stat != 0] # remove zero elements

data_min = np.min(stat)
data_max = np.max(stat)

# normalize to range 0-1
norm = 1 / data_max
stat = stat * norm

In [7]:
data_array = np.linspace(data_min, data_max, len(stat))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_r = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_r_axis_label)

# set y-limits
if ylim_r is not None:
    plot_r.set_ylim(ylim_r)

if ylim_z is not None:
    plot_z.set_ylim(ylim_z)

In [19]:
# add data
if use_approx:
    x_new = np.linspace(velocity_min, velocity_max, len(stat))

    coefs = poly.polyfit(slices, stat, approx_degree)
    ffit = poly.polyval(x_new, coefs)
    plot_r.plot(x_new, ffit)
    plot_r.plot(slices, stat, linestyle = 'None', marker='.')
else:
    plot_r.plot(slices, stat)

plot.show()